In [ ]:
!git clone https://github.com/openai/CLIP
!pip install ftfy regex tqdm omegaconf pytorch-lightning torch torchvision transformers
!wget https://www.dropbox.com/s/2oxu7hw0y9fwdqs/M-BERT-Base-69-ViT%20Linear%20Weights.pkl

Cloning into 'CLIP'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 185 (delta 7), reused 13 (delta 4), pack-reused 168
Receiving objects: 100% (185/185), 8.90 MiB | 29.60 MiB/s, done.
Resolving deltas: 100% (89/89), done.
     |████████████████████████████████| 64 kB 2.0 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
     |████████████████████████████████| 526 kB 47.9 MB/s 
     |████████████████████████████████| 3.4 MB 59.0 MB/s 
     |████████████████████████████████| 596 kB 54.5 MB/s 
     |████████████████████████████████| 112 kB 57.8 MB/s 
     |████████████████████████████████| 332 kB 62.2 MB/s 
     |████████████████████████████████| 829 kB 58.6 MB/s 
     |████████████████████████████████| 133 kB 52.4 MB/s 
     |████████████████████████████████| 1.1 MB 55.2 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 8

--2022-01-15 20:07:56--  https://www.dropbox.com/s/2oxu7hw0y9fwdqs/M-BERT-Base-69-ViT%20Linear%20Weights.pkl
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/2oxu7hw0y9fwdqs/M-BERT-Base-69-ViT%20Linear%20Weights.pkl [following]
--2022-01-15 20:07:56--  https://www.dropbox.com/s/raw/2oxu7hw0y9fwdqs/M-BERT-Base-69-ViT%20Linear%20Weights.pkl
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb1f5933557f2983bd649085427.dl.dropboxusercontent.com/cd/0/inline/Bd34uXOMw6RC_3fOez4F4rISd_wSMgtZkizKtyJewbmIJXYhQc_BvZwjq3ZN5o0JKDnyJRid_MRa1IuGIhSbbHjYePm-KLhro6T4Jd2asCLNwaq32H5jktddkqCRp_ynoe1TLBm9Tmns3cYb4GbfJ9Dp/file# [following]
--2022-01-15 20:07:57--  https://ucb1f5933557f2983bd649085427.dl.dropboxusercontent.com/cd/0/inline/Bd3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# with zipfile.ZipFile('/content/drive/MyDrive/ivision/parser_video_dtp_5_720p.zip', 'r') as zip_ref:
#     zip_ref.extractall('/content/drive/MyDrive/ivision/')

In [ ]:
dtp_ids = [
    # 1,
    2,
    # 4,
    5,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    21,
    # 22,
    24,
    27,
    33,
    36,
    50]

In [ ]:

import torch
from CLIP import clip
from PIL import Image
from torch import nn
device = "cuda" if torch.cuda.is_available() else "cpu"
import time
all_time = 0
model, preprocess = clip.load("ViT-B/32", device=device,jit=False)
import pickle

import torch
import transformers


class MultilingualClip(torch.nn.Module):
    def __init__(self, model_name, tokenizer_name, head_name, weights_dir='/content/'):
        super().__init__()
        self.model_name = model_name
        self.tokenizer_name = tokenizer_name
        self.head_path = weights_dir + head_name

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(tokenizer_name)
        self.transformer = transformers.AutoModel.from_pretrained(model_name)
        self.clip_head = torch.nn.Linear(in_features=768, out_features=640)
        self._load_head()

    def forward(self, txt):
        txt_tok = self.tokenizer(txt, padding=True, return_tensors='pt')
        
        embs = self.transformer(**txt_tok)[0]
        att = txt_tok['attention_mask']
        embs = (embs * att.unsqueeze(2)).sum(dim=1) / att.sum(dim=1)[:, None]
        return self.clip_head(embs)

    def _load_head(self):
        with open(self.head_path, 'rb') as f:
            lin_weights = pickle.loads(f.read())
        self.clip_head.weight = torch.nn.Parameter(torch.tensor(lin_weights[0]).float().t())
        self.clip_head.bias = torch.nn.Parameter(torch.tensor(lin_weights[1]).float())


AVAILABLE_MODELS = { 
    'M-BERT-Base-ViT-B': {
        'model_name': 'M-CLIP/M-BERT-Base-ViT-B',
        'tokenizer_name': 'M-CLIP/M-BERT-Base-ViT-B',
        'head_name': 'M-BERT-Base-69-ViT Linear Weights.pkl'
    },
}


def load_model(name):
    config = AVAILABLE_MODELS[name]
    return MultilingualClip(**config)



device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
text_model = load_model('M-BERT-Base-ViT-B')

# text_model.to(device)
model.to(device)





def max_cosine_images(image_paths,label):
  """
  image_paths: массив путей к файлам в которых ищем
  label : текстовое описание того что ищем 
  """
  cos = nn.CosineSimilarity(dim=1, eps=1e-32)
  
  

  embed = text_model(label)
  text_features = embed[:,:512]
  
  with torch.no_grad():
    max = 0
    max_score_image = None

    for image_p in image_paths:
      
      
      image= preprocess(Image.open(image_p)).unsqueeze(0).to(device)
  

      image_features = model.encode_image(image)

      
      
        

      if cosine>max:
        max = cosine
        path = image_p
        max_score_image  = image
        

    
    return max, max_score_image,path 
    

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 167MiB/s]


Downloading:   0%|          | 0.00/297 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [ ]:
class MyTextEmbedder:
    def __init__(self, labels, text_model):
        self.model = text_model
        self.text_dict = dict()
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        with torch.no_grad():
            for txt in labels:
                embed = self.model(txt)
                text_features = embed[:,:512]
                text_features = text_features.to(device)
                self.text_dict[txt] = text_features
    def get_cos(self, image_features, text_features):
        cos = nn.CosineSimilarity(dim=1, eps=1e-32)
        cosine=cos(image_features, text_features)
        return cosine
    def get_max_sim(self, image_features):
        max = 0
        label = ''
        for txt in self.text_dict.keys():
            text_features = self.text_dict[txt]
            cosine = self.get_cos(image_features, text_features)
            if cosine>max:
                max = cosine
                label = txt
        return max, label
    # def check_cos(self, image_features)


In [ ]:
my_text_classes = [ 'crossroad','Normal traffic']
my_text_classes2 = ['White police car in the traffic', 'No police car on the road']
my_text_classes3 = ['Fire truck', 'No fire truck']
my_text_classes4 = ['Spetial Car with flashing lights on the road', 'No Car with flashing lights']
text_embedder = MyTextEmbedder(my_text_classes,text_model)
police_embedder = MyTextEmbedder(my_text_classes2,text_model)
fire_embedder = MyTextEmbedder(my_text_classes3,text_model)
flash_embedder = MyTextEmbedder(my_text_classes4,text_model)

def check_cos(image_features, text_features):
    cos = nn.CosineSimilarity(dim=1, eps=1e-32)
    cosine=cos(image_features, text_features)
    return cosine 
def predict_img_emb(model, image_path):
    image= preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    image_features = model.encode_image(image)
    return image_features
import os

def predict_video(video_path):
    # labels_dict = { 'vehicle accident': 0,
    #                'not vehicle accident': 0}
    labels_dict = {name: 0 for name in my_text_classes}
    labels_dict_2 = {name: 0 for name in my_text_classes2}
    labels_dict_3 = {name: 0 for name in my_text_classes3}
    labels_dict_4 = {name: 0 for name in my_text_classes4}
    cur_img = 0
    # police_car_cos =
    for imgfile in os.listdir(video_path):
        cur_img += 1
        if cur_img % 10 != 0:
            continue
        img_path = os.path.join(video_path, imgfile)
        img_emb = predict_img_emb(model, img_path)
        max_val, max_label = text_embedder.get_max_sim(img_emb)

        labels_dict[max_label] += 1

        max_val, max_label = police_embedder.get_max_sim(img_emb)

        labels_dict_2[max_label] += 1

        max_val, max_label = fire_embedder.get_max_sim(img_emb)

        labels_dict_3[max_label] += 1

        max_val, max_label = flash_embedder.get_max_sim(img_emb)

        labels_dict_4[max_label] += 1
        print(check_cos(img_emb, text_embedder.text_dict['crossroad']))
    super_max_label = ''
    max_cnt = 0
    for k in labels_dict.keys():
        if labels_dict[k] > max_cnt:
            max_cnt = labels_dict[k]
            super_max_label = k
    
    # if labels_dict[my_text_classes[0]] > 10:
    #     return my_text_classes[0]
    labels = []
    if labels_dict[my_text_classes[0]] >= 1/3 * (labels_dict[my_text_classes[0]] + labels_dict[my_text_classes[1]]):
        labels.append(my_text_classes[0])
    else:
        labels.append(my_text_classes[1])
    # labels.append(super_max_label)
    if labels_dict_2[my_text_classes2[0]] > 4:
        labels.append(my_text_classes2[0])
    else:
        labels.append(my_text_classes2[1])

    if labels_dict_3[my_text_classes3[0]] > 4:
        labels.append(my_text_classes3[0])
    else:
        labels.append(my_text_classes3[1])
    
    if labels_dict_4[my_text_classes4[0]] > 4:
        labels.append(my_text_classes4[0])
    else:
        labels.append(my_text_classes4[1])
    print(labels_dict, labels_dict_2, labels_dict_3, labels_dict_4)
    return labels



In [ ]:
acc = 0
cnt = 0
for video in sorted(list(os.listdir('/content/drive/MyDrive/ivision/parser_video_dtp_5_720p'))):
    video_label = predict_video(os.path.join('/content/drive/MyDrive/ivision/parser_video_dtp_5_720p', video))
    print(video, ':')
    true_label = ''
    
    if int(video) in dtp_ids:
        print('True label:', my_text_classes[0])
        true_label = my_text_classes[0]
    else:
        print('True label:', my_text_classes[1])
        true_label = my_text_classes[1]
    print('Predicted label:', video_label)
    acc += int(video_label[0] == true_label)

    print('----------------',cnt,'/', 50, '----------------------')

    cnt += 1
print(acc / 50 * 100)

tensor([0.2515], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2536], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2546], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2435], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2540], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2583], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2539], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2516], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2517], device='cuda:0', grad_fn=<DivBackward0>)
tensor([0.2479], device='cuda:0', grad_fn=<DivBackward0>)
{'crossroad': 10, 'Normal traffic': 0} {'White police car in the traffic': 5, 'No police car on the road': 5} {'Fire truck': 0, 'No fire truck': 10} {'Spetial Car with flashing lights on the road': 2, 'No Car with flashing lights': 8}
1 :
True label: Normal traffic
Predicted label: ['crossroad', 'White police car in the traffic', 'No fire truck', 'No Car with flashing lights']
---------------- 0 / 50 ----------------

KeyboardInterrupt: ignored

In [ ]:
def max_cosine_text(image_path,labels):
    """
    image_path: путь к изображению для которго ищем
    text: текстовые метки для которы ищем
    """
    cos = nn.CosineSimilarity(dim=1, eps=1e-32)
    # print(1)
    t = time.time()
    image= preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    print('Время обработки: ', time.time() - t)

  
    
    with torch.no_grad():
        max = 0
        label = ''
        image_features = model.encode_image(image)
        print('После картинки: ', time.time() - t)
        for txt in labels:
            t = time.time()
            embed = text_model(txt)
            print('После текста: ', time.time() - t)
            text_features = embed[:,:512]
            text_features = text_features.to(device)
            
            cosine=cos(image_features,text_features)
            
            if cosine>max:
                max = cosine
                label = txt
        

    
    return max,label

# Multiply text input

1 :
True label: vehicle accident
Predicted label: have not emergency car
10 :
True label: vehicle accident
Predicted label: have not emergency car
11 :
True label: vehicle accident
Predicted label: have not emergency car
12 :
True label: vehicle accident
Predicted label: have not emergency car


KeyboardInterrupt: ignored

In [ ]:
t3 = time.time()
max_cosine_text('/content/2008-03-11_Bicyclist_in_Carrboro.jpg',[ 'vehicle accident','not vehicle accident'])
print(time.time() - t3)


Время обработки:  0.060974836349487305
После картинки:  0.07175993919372559
После текста:  0.06282615661621094
После текста:  0.060387611389160156
0.19846367835998535


In [ ]:
all_time / 100

0.20537013053894043

# Multiply image input

In [ ]:
_ , _ ,path =max_cosine_images(['/content/Prm_VJ_fig1_featureTypesWithAlpha.png','/content/2008-03-11_Bicyclist_in_Carrboro.jpg'],'велосипед')

print(path)

/content/2008-03-11_Bicyclist_in_Carrboro.jpg
